In [1]:
%matplotlib auto
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator

from skimage import io
import os
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from skimage.io import imread, imsave
from matplotlib import pyplot as plt 

from PIL import Image
import imutils

Using matplotlib backend: Qt5Agg


In [10]:
import requests
address = 'http://192.168.0.30:8080/shot.jpg'
colors = {}
ohms_list = []

color_bands = {"Black": 0, "Brown": 1, "Red": 2, "Orange": 3, "Yellow": 4, "Green": 5, "Blue": 6, "Violet": 7, "Silver": 8, "White": 9}
color_multipliers = {"Black": 1, "Brown": 10, "Red": 100, "Orange": 1000, "Yellow": 10000, "Green": 100000, "Blue":1000000 , "Violet": 10000000, "Silver": 100000000, "White": 100000000}


lower_blue = np.array([80, 50, 50]) 
upper_blue = np.array([106, 250, 150])

lower_green = np.array([40, 40, 40]) 
upper_green = np.array([80, 115, 255]) 

lower_red = np.array([0, 85, 0]) 
upper_red = np.array([5, 255, 200])

lower_yellow = np.array([19, 87, 136]) 
upper_yellow = np.array([24, 255, 255])

lower_orange = np.array([5, 100, 160]) 
upper_orange = np.array([15, 255, 255]) 

lower_violet = np.array([[115, 0, 90]])
upper_violet = np.array([175, 255, 255])

lower_black = np.array([[0, 0, 0]])
upper_black = np.array([179, 255, 40])

lower_brown = np.array([7, 40, 85]) 
upper_brown = np.array([15, 255, 145]) 

lower_silver= np.array([[15, 0, 120]])
upper_silver = np.array([179, 30, 135])

temp_start = (900, 545)
temp_end = (1000, 550)

#Find the area that is bigger than the area threshold and store the color and its x-coordinate
def find_color(contour, color):
    for c in contour:
        area = cv2.contourArea(c)
        if area > area_limit:
            print(color, area)
            M = cv2.moments(c)
            cx = int(M["m10"]/ M["m00"])
            cy = int(M["m01"]/ M["m00"])
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),1)
            print("x: ", x)
            if x not in colors:
                colors[x] = color
                
                
while True:
        
    img_resp = requests.get(address)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    frame = cv2.imdecode(img_arr, -1)
    height, width, channels = frame.shape
    show_frame = frame
    
    cv2.bilateralFilter(frame, 5, 800, 800)
    
    cv2.rectangle(frame, (int(width*0.5)-45, int(height*0.5)), (int(width*0.5)+50,int(height*0.5)-5) , (255,255,255), 1)
    #cv2.rectangle(frame, temp_start, temp_end, (255,255,255), 1)
    
    frame = frame[int(height*0.5):int(height*0.5)+10, int(width*0.5)-50:int(width*0.5)+50]
    #frame = frame[550:560, 900:1000]
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    

    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_orange = cv2.inRange(hsv, lower_orange, upper_orange)
    mask_violet = cv2.inRange(hsv, lower_violet, upper_violet)
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
    mask_silver = cv2.inRange(hsv, lower_silver, upper_silver)
    
    
    contour1 = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour1 = imutils.grab_contours(contour1)
    
    contour2 = cv2.findContours(mask_green, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour2 = imutils.grab_contours(contour2)
    
    contour3 = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour3 = imutils.grab_contours(contour3)
    
    contour4 = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour4 = imutils.grab_contours(contour4)
    
    contour5 = cv2.findContours(mask_orange, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour5 = imutils.grab_contours(contour5)
    
    contour6 = cv2.findContours(mask_violet, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour6 = imutils.grab_contours(contour6)
    
    contour7 = cv2.findContours(mask_black, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour7 = imutils.grab_contours(contour7)
    
    contour8 = cv2.findContours(mask_brown, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour8 = imutils.grab_contours(contour8)
    
    contour9 = cv2.findContours(mask_silver, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour9 = imutils.grab_contours(contour9)
    
    
    area_limit = 20
                
    find_color(contour1, 'Blue')
    find_color(contour2, 'Green')    
    find_color(contour3, 'Red')
    find_color(contour4, 'Yellow')
    find_color(contour5, 'Orange')
    find_color(contour6, 'Violet')
    find_color(contour7, 'Black')
    find_color(contour8, 'Brown')
    find_color(contour9, 'Silver')
    
    
    #Calculate and display ohmic value
    colors_list = []
    for color in colors.keys():
             colors_list.append(color)

    colors_list = sorted(colors_list)
    print(colors_list, " sorted and before pop")
    
    
    #If two of the same colors are almost on the same x-position, chose only one.
    #Remove colors that are near the same X-coordinate (e.g. if there are two 
    #colors detected on the same stripe)
    
     
    for i in range(len(colors_list)):
        try:
            if colors_list[i] - (colors_list[i+1]) < 0:
                print("reached if pop")
                test = colors_list.pop(i+1)
                print("popped: ", test)

               
        except IndexError:
            print("index error")
            
    print(colors_list, " sorted and after pop")
    
    try:
        if len(colors_list) <3:
            print("reached if")
            pass

        else:
            color1 = colors[colors_list[0]]
            color2 = colors[colors_list[1]]
            color3 = colors[colors_list[2]]
            print("Colors: ", color1, color2, " * ", color3)
            colors = {}
            
            
        combined = int(str(color_bands[color1]) + str(color_bands[color2]))

        ohms = combined*color_multipliers[color3]

        if ohms < 10**3:
            ohms = (combined*color_multipliers[color3])
            disp = str(ohms)
        if ohms < 10**6 and ohms > 10**3:
            ohms = (combined*color_multipliers[color3])/(10**3)
            disp = str(ohms) + "k"
        if ohms >= 10**6:
            ohms = (combined*color_multipliers[color3])/(10**6)
            disp = str(ohms) + "M"
        
        
        #Filter out value noise, list -> most frequent element gets printed to reduce twitching    
        ohms_list.append(disp)

        if len(ohms_list) > 30:
            print("list reset")
            ohms_list = [ohms]

        if disp == '0':
            print("skip 0's")
            pass

        else:
            disp = str(max(ohms_list, key=ohms_list.count))
            print(ohms_list) 

        disp_colors = str("Colors: " + color1 + " " + color2 + " * " + color3)

        cv2.putText(show_frame,disp + ' ohm',(0,int(height*0.9)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        print("output: " + disp)
        print("_______________________")
        cv2.putText(show_frame,disp_colors,(int(width*0.75),int(height*0.9)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


    except NameError:
        print("NameError")
                
    
    print(show_frame.shape)
    cv2.imshow("result",show_frame)
    
    #Close window
    key = cv2.waitKey(1) 
    if key == 27:
        break
    
        print(colors)
    
        

cv2.destroyAllWindows()
cv2.waitKey(1)

[]  sorted and before pop
[]  sorted and after pop
reached if
['4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[]  sorted and before pop
[]  sorted and after pop
reached if
['4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[]  sorted and before pop
[]  sorted and after pop
reached if
['4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[]  sorted and before pop
[]  sorted and after pop
reached if
['4.7k', '4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[]  sorted and before pop
[]  sorted and after pop
reached if
['4.7k', '4.7k', '4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
Brown 843.5
x:  0
[0]  sorted and before pop
index error
[0]  sorted and after pop
reached if
['4.7k', '4.7k', '4.7k', '4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[0]  sorted and before pop
index error
[0]  sorted and after pop
reached if
['4.7k', '4.7k', '4.7k', '4.

[10]  sorted and before pop
index error
[10]  sorted and after pop
reached if
[3.2, '3.2k']
output: 3.2
_______________________
(1080, 1920, 3)
Red 80.5
x:  75
Red 40.0
x:  40
Yellow 100.0
x:  14
[10, 14, 40, 75]  sorted and before pop
reached if pop
popped:  14
reached if pop
popped:  75
index error
index error
[10, 40]  sorted and after pop
reached if
[3.2, '3.2k', '3.2k']
output: 3.2k
_______________________
(1080, 1920, 3)
[10, 14, 40, 75]  sorted and before pop
reached if pop
popped:  14
reached if pop
popped:  75
index error
index error
[10, 40]  sorted and after pop
reached if
[3.2, '3.2k', '3.2k', '3.2k']
output: 3.2k
_______________________
(1080, 1920, 3)
Silver 54.0
x:  0
[0, 10, 14, 40, 75]  sorted and before pop
reached if pop
popped:  10
reached if pop
popped:  40
index error
index error
index error
[0, 14, 75]  sorted and after pop
Colors:  Silver Yellow  *  Red
[3.2, '3.2k', '3.2k', '3.2k', '8.4k']
output: 3.2k
_______________________
(1080, 1920, 3)
[]  sorted and befo

x:  94
Yellow 70.0
x:  33
Orange 20.5
x:  91
Violet 80.0
x:  71
Brown 52.0
x:  8
[8, 33, 71, 91, 94]  sorted and before pop
reached if pop
popped:  33
reached if pop
popped:  91
index error
index error
index error
[8, 71, 94]  sorted and after pop
Colors:  Brown Violet  *  Red
[3.2, '3.2k', '3.2k', '3.2k', '8.4k', '8.4k', '8.4k', '140', '140', '140', '140', '140', '110.0M', '110.0M', '4.2k', '4.2k', '10', '10', '10', '1.1k', '410', '410', '4.7k', '4.7k', '140.0M', '1.7k']
output: 140
_______________________
(1080, 1920, 3)
Red 40.0
x:  94
Yellow 59.5
x:  34
Violet 75.5
x:  70
Brown 61.0
x:  6
[6, 34, 70, 94]  sorted and before pop
reached if pop
popped:  34
reached if pop
popped:  94
index error
index error
[6, 70]  sorted and after pop
reached if
[3.2, '3.2k', '3.2k', '3.2k', '8.4k', '8.4k', '8.4k', '140', '140', '140', '140', '140', '110.0M', '110.0M', '4.2k', '4.2k', '10', '10', '10', '1.1k', '410', '410', '4.7k', '4.7k', '140.0M', '1.7k', '1.7k']
output: 140
_______________________

Orange 95.5
x:  55
Black 55.0
x:  38
Brown 60.0
x:  4
[4, 5, 38, 39, 55, 57]  sorted and before pop
reached if pop
popped:  5
reached if pop
popped:  39
reached if pop
popped:  57
index error
index error
index error
[4, 38, 55]  sorted and after pop
Colors:  Brown Black  *  Orange
[140.0, '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '4.7k', '440.0M', '440.0M', '440.0M', '470.0M', '470.0M', '4.7k', '4.7k', '10.0k', '10.0k', '10.0k']
output: 4.7k
_______________________
(1080, 1920, 3)
Orange 91.0
x:  54
Black 50.5
x:  36
Brown 52.5
x:  2
[2, 36, 54]  sorted and before pop
reached if pop
popped:  36
index error
index error
[2, 54]  sorted and after pop
reached if
[140.0, '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '4.7k', '440.0M', '440.0M', '440.0M', '470.0M', '470.0M', '4.7k', '4.7k', '10.0k', '10.0k', '10.0k', '10.0k']
output: 4.7k
_______________________
(1080, 1920, 3)
Orange 90.0
x:  55
Black 52.0
x:  37
Brown 59.0
x:  3
[2, 3, 36, 37, 54, 55]  sorted and before pop
reached if pop
poppe

Orange 249.0
x:  51
Orange 96.0
x:  20
[20, 51]  sorted and before pop
reached if pop
popped:  51
index error
[20]  sorted and after pop
reached if
[830, '11.0k', '11.0k', '31.0k', '31.0k', '34.0k', '34.0k']
output: 11.0k
_______________________
(1080, 1920, 3)
Orange 89.0
x:  76
Orange 110.0
x:  53
Orange 88.0
x:  21
[20, 21, 51, 53, 76]  sorted and before pop
reached if pop
popped:  21
reached if pop
popped:  53
index error
index error
index error
[20, 51, 76]  sorted and after pop
Colors:  Orange Orange  *  Orange
[830, '11.0k', '11.0k', '31.0k', '31.0k', '34.0k', '34.0k', '33.0k']
output: 11.0k
_______________________
(1080, 1920, 3)
Orange 209.5
x:  55
Orange 72.5
x:  24
[24, 55]  sorted and before pop
reached if pop
popped:  55
index error
[24]  sorted and after pop
reached if
[830, '11.0k', '11.0k', '31.0k', '31.0k', '34.0k', '34.0k', '33.0k', '33.0k']
output: 11.0k
_______________________
(1080, 1920, 3)
Orange 110.0
x:  75
Orange 103.0
x:  55
Orange 82.5
x:  24
[24, 55, 75]  s

...

[24, 66]  sorted and before pop
reached if pop
popped:  66
index error
[24]  sorted and after pop
reached if
[110, '110', '110', '110', '110', '110', '110', '110']
output: 110
_______________________
(1080, 1920, 3)
[24, 66]  sorted and before pop
reached if pop
popped:  66
index error
[24]  sorted and after pop
reached if
[110, '110', '110', '110', '110', '110', '110', '110', '110']
output: 110
_______________________
(1080, 1920, 3)
[24, 66]  sorted and before pop
reached if pop
popped:  66
index error
[24]  sorted and after pop
reached if
[110, '110', '110', '110', '110', '110', '110', '110', '110', '110']
output: 110
_______________________
(1080, 1920, 3)
[24, 66]  sorted and before pop
reached if pop
popped:  66
index error
[24]  sorted and after pop
reached if
[110, '110', '110', '110', '110', '110', '110', '110', '110', '110', '110']
output: 110
_______________________
(1080, 1920, 3)
[24, 66]  sorted and before pop
reached if pop
popped:  66
index error
[24]  sorted and after 

(1080, 1920, 3)
Orange 386.5
x:  0
Black 151.0
x:  60
Brown 21.0
x:  50
[0, 50, 60]  sorted and before pop
reached if pop
popped:  50
index error
index error
[0, 60]  sorted and after pop
reached if
[110, '110', '110', '110', '300', '300']
output: 110
_______________________
(1080, 1920, 3)
Orange 843.5
x:  0
[0, 50, 60]  sorted and before pop
reached if pop
popped:  50
index error
index error
[0, 60]  sorted and after pop
reached if
[110, '110', '110', '110', '300', '300', '300']
output: 110
_______________________
(1080, 1920, 3)
Black 119.0
x:  11
Brown 26.5
x:  1
Silver 27.0
x:  66
[0, 1, 11, 50, 60, 66]  sorted and before pop
reached if pop
popped:  1
reached if pop
popped:  50
reached if pop
popped:  66
index error
index error
index error
[0, 11, 60]  sorted and after pop
Colors:  Orange Black  *  Black
[110, '110', '110', '110', '300', '300', '300', '30']
output: 110
_______________________
(1080, 1920, 3)
[]  sorted and before pop
[]  sorted and after pop
reached if
[110, '110'

x:  0
Silver 29.5
x:  41
[0, 36, 41, 52]  sorted and before pop
reached if pop
popped:  36
reached if pop
popped:  52
index error
index error
[0, 41]  sorted and after pop
reached if
[30, '30', '30', '200.0M', '200.0M', '200.0M', '200.0M', '200.0M']
output: 200.0M
_______________________
(1080, 1920, 3)
Black 724.5
x:  0
[0, 36, 41, 52]  sorted and before pop
reached if pop
popped:  36
reached if pop
popped:  52
index error
index error
[0, 41]  sorted and after pop
reached if
[30, '30', '30', '200.0M', '200.0M', '200.0M', '200.0M', '200.0M', '200.0M']
output: 200.0M
_______________________
(1080, 1920, 3)
Black 772.5
x:  0
[0, 36, 41, 52]  sorted and before pop
reached if pop
popped:  36
reached if pop
popped:  52
index error
index error
[0, 41]  sorted and after pop
reached if
[30, '30', '30', '200.0M', '200.0M', '200.0M', '200.0M', '200.0M', '200.0M', '200.0M']
output: 200.0M
_______________________
(1080, 1920, 3)
Black 299.0
x:  61
Black 346.0
x:  0
[0, 36, 41, 52, 61]  sorted and 

Red 35.0
x:  81
Red 639.5
x:  0
[0, 81]  sorted and before pop
reached if pop
popped:  81
index error
[0]  sorted and after pop
reached if
[0, '0', '0', '0', '0', '0', '0', '0', '0', '0', '2.0k', '2.0k']
output: 0
_______________________
(1080, 1920, 3)
Red 144.0
x:  81
Red 624.0
x:  0
[0, 81]  sorted and before pop
reached if pop
popped:  81
index error
[0]  sorted and after pop
reached if
[0, '0', '0', '0', '0', '0', '0', '0', '0', '0', '2.0k', '2.0k', '2.0k']
output: 0
_______________________
(1080, 1920, 3)
Red 767.0
x:  0
[0, 81]  sorted and before pop
reached if pop
popped:  81
index error
[0]  sorted and after pop
reached if
[0, '0', '0', '0', '0', '0', '0', '0', '0', '0', '2.0k', '2.0k', '2.0k', '2.0k']
output: 0
_______________________
(1080, 1920, 3)
Red 786.0
x:  0
[0, 81]  sorted and before pop
reached if pop
popped:  81
index error
[0]  sorted and after pop
reached if
[0, '0', '0', '0', '0', '0', '0', '0', '0', '0', '2.0k', '2.0k', '2.0k', '2.0k', '2.0k']
output: 0
_______

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))